# Gemini Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an Gemini API key. For testing purposes, you can generate a new account, and use your evaluation test key (no credit card required!).

Once you have acquired your key, set it to the environment variable `GOOGLE_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and Gemini, you will need to install the following packages:

```shell
pip install unitycatalog-gemini
```

In [3]:
import os

assert "GOOGLE_API_KEY" in os.environ, (
    "Please set the GOOGLE_API_KEY environment variable to your Gemini API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [ ]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.ai.gemini.toolkit import UCFunctionToolkit
from unitycatalog.client import ApiClient, Configuration

In [8]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define a function and register it to Unity Catalog

In this next section, we'll be defining a placeholder Python function and creating it within Unity Catalog so that it can be retrieved and used as a tool by gemini's Claude model. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [9]:
def fetch_weather(location: str) -> str:
    """
    Fetches the current weather in celsius for a given location.

    Args:
        location (str): The location to fetch the weather for.

    Returns:
        str: The current weather in celsius for the given location.
    """

    return "88.2 F"

In [10]:
client.create_python_function(func=fetch_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(name='fetch_weather', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='STRING', type_json='{"name": "location", "type": "string", "nullable": false, "metadata": {"comment": "The location to fetch the weather for."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The location to fetch the weather for.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='return "88.2 F"', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', is_null_call=False, security_type='DEFINER', specific_name='fetch_weather', comment='Fetches the current weather in celsius for a given location.', properties='null', full_name='AIC

## Create a Toolkit instance of the function(s)

Now that the function has been created within Unity Catalog, we can use the `unitycatalog-gemini` package to create a toolkit instance that gemini will 'understand' as a valid tool in its APIs. 

In [11]:
toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.fetch_weather"], client=client)

tools = toolkit.generate_callable_tool_list()

name='fetch_weather' catalog_name='AICatalog' schema_name='AISchema' input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='STRING', type_json='{"name": "location", "type": "string", "nullable": false, "metadata": {"comment": "The location to fetch the weather for."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The location to fetch the weather for.')]) data_type=<ColumnTypeName.STRING: 'STRING'> full_data_type='STRING' return_params=None routine_body='EXTERNAL' routine_definition='return "88.2 F"' routine_dependencies=None parameter_style='S' is_deterministic=True sql_data_access='NO_SQL' is_null_call=False security_type='DEFINER' specific_name='fetch_weather' comment='Fetches the current weather in celsius for a given location.' properties=None full_name='AICatalog.AISchema.fetch_weather' ow

In [12]:
tools

## Call Gemini with tools

With our toolkit instance ready, we can now pass in our tools to a call to Gemini, giving Gemini the ability to request for 'answers' from tools that have been provided with the request.

In [13]:
# Interface with Gemini via their SDK
from google.generativeai import GenerativeModel

multi = "What's the weather in Nome, AK and in Death Valley, CA?"


model = GenerativeModel(model_name="gemini-2.0-flash-exp", tools=tools)

chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message("What's the weather in Nome, AK and in Death Valley, CA?")
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The weather in Nome, AK is 88.2 F, and the weather in Death Valley, CA is 88.2 F.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "avg_logp

## Show Details of Tool Calls

In [15]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])  # noqa: T201
    print("-" * 80)  # noqa: T201

user -> [{'text': "What's the weather in Nome, AK and in Death Valley, CA?"}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'fetch_weather', 'args': {'location': 'Nome, AK'}}}, {'function_call': {'name': 'fetch_weather', 'args': {'location': 'Death Valley, CA'}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'fetch_weather', 'response': {'result': '{"format": "SCALAR", "value": "88.2 F"}'}}}, {'function_response': {'name': 'fetch_weather', 'response': {'result': '{"format": "SCALAR", "value": "88.2 F"}'}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'The weather in Nome, AK is 88.2 F, and the weather in Death Valley, CA is 88.2 F.\n'}]
--------------------------------------------------------------------------------


## Manually execute function calls

In [ ]:
from google.generativeai.types import content_types

from unitycatalog.ai.gemini.utils import generate_tool_call_messages, get_function_calls

history = []
question = "What's the weather in Nome, AK and in Death Valley, CA?"


content = content_types.to_content(question)
if not content.role:
    content.role = "user"

history.append(content)

response = model.generate_content(history)
while function_calls := get_function_calls(response):
    history, function_calls = generate_tool_call_messages(
        model=model, response=response, conversation_history=history
    )

    response = model.generate_content(history)

response